In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import pandas as pd
train_path = tf.keras.utils.get_file("train.csv","https://datahack.analyticsvidhya.com/contest/game-of-deep-learning/download/train-file", extract = True)
data_root = pathlib.Path(train_path)
p = pathlib.Path(r'C:\Users\Arnab Sinha\Desktop\BAAPFOLDER\Datasets\Game of DL - AV\images')
classes = {'Cargo':0,'Military':1,'Carrier':2,'Cruise':3,'Tankers':4}
traincsv = pd.read_csv(r'C:\Users\Arnab Sinha\Desktop\BAAPFOLDER\Datasets\Game of DL - AV\train.csv')
ail = list(traincsv['category'].values)
ail = [a-1 for a in ail]
image_csv = list(traincsv['image'].values)
# for x in range(len(ail)):
#     print(image_csv[x],' ',ail[x])
print(len(ail))
oka = [item.split('.jpg') for item in image_csv]
oka = [oka[item][0] for item in range(len(oka))]
aip_train = ['C:\\Users\\Arnab Sinha\\Desktop\\BAAPFOLDER\\Datasets\\Game of DL - AV\\images\\' + item + '.jpg' for item in oka]
type(aip_train[0])

aip = list(p.glob('*/'))
aip = [str(path) for path in aip]

path_ds = tf.data.Dataset.from_tensor_slices(aip_train)
# print(path_ds.take(4))

# print(path_ds)

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [160,160])
  image/=255.0
  return image

AUTOTUNE = tf.data.experimental.AUTOTUNE
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

# sample = tf.io.read_file('C:\\Users\\Arnab Sinha\\Desktop\\BAAPFOLDER\\Datasets\\Game of DL - AV\\images\\1001524.jpg')
# sample = tf.image.decode_jpeg(sample,channels = 3)
# print(sample.shape)

import re

image_n = [re.findall("[0-9][0-9]*",str) for str in aip]
image_no = []
for item in image_n:
    image_no.append(item[0])
image_no

diff = list(set(image_no)-set(oka))

label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(ail, tf.int64))

image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

print(image_label_ds)

image_count = len(aip_train)

image_count

ds = tf.data.Dataset.from_tensor_slices((aip_train, ail))

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds

BATCH_SIZE = 32

ds = image_label_ds.cache()
ds = ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds

mobile_net = tf.keras.applications.MobileNetV2(input_shape=(160, 160, 3), include_top=False)
mobile_net.trainable=False

def change_range(image,label):
  return 2*image-1, label

keras_ds = ds.map(change_range)

# The dataset may take a few seconds to start, as it fills its shuffle buffer.
image_batch, label_batch = next(iter(keras_ds))

label_names = [0,1,2,3,4]
model = tf.keras.Sequential([
  mobile_net,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(len(label_names))])

logit_batch = model(image_batch).numpy()

print("min logit:", logit_batch.min())
print("max logit:", logit_batch.max())
print()

print("Shape:", logit_batch.shape)

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

steps_per_epoch=tf.math.ceil(len(aip_train)/BATCH_SIZE).numpy()
steps_per_epoch

model.fit(ds, epochs=1, steps_per_epoch=20)

6252
<ZipDataset shapes: ((160, 160, 3), ()), types: (tf.float32, tf.int64)>
min logit: -1.3205826
max logit: 2.089417

Shape: (32, 5)
20/20 [==============================] - 162s 8s/step - loss: 3.4132 - accuracy: 0.2891
